In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Counter

In [10]:
# clustering of words used 35 clusters
word_class = pd.read_csv('word_class35.csv',header=None).dropna()

In [11]:
word_class.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247500 entries, 0 to 247501
Data columns (total 2 columns):
0    247500 non-null object
1    247500 non-null int64
dtypes: int64(1), object(1)
memory usage: 5.7+ MB


In [12]:
index = word_class[0].apply(lambda x: ' '.join(x.split('_')))
index.head()

0      what
1        we
2      know
3       and
4    do not
Name: 0, dtype: object

In [7]:
word_class['0'] = index
word_class.set

'what'

In [51]:
word_class.head()

,1
0,
what,4
we,32
know,32
and,32
do_not,32


In [64]:
# change dataframe to dictionary with words as keys and topic as values
word_topic = word_class[1].T.to_dict()

In [66]:
word_topic['we'],word_topic['what']

(32, 4)

In [2]:
# table with id,year,text,keyword columns
df = pd.read_csv('./Processed_data/norm_text_keyword.csv')

/Users/shunling/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
df.head(2)

,Unnamed: 0,Id,year,text,keywords
0,0,24877594,2014.0,what we know and do not know about the cannab...,"cannabinoid_receptor_2,endocannabinoid,immune_..."
1,1,16889837,2006.0,allosteric agonist of 7tm receptor expand th...,NaN


In [84]:

def predict_topic(abstract):
    """return top10 topics in the abstract"""
    text = abstract.split()
    topics = Counter()
    for word in text:
        topics[word_topic[word]] += 1
        
    top10 = list(dict(topics.most_common(10)).keys())
    return top10    

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323870 entries, 0 to 323869
Data columns (total 5 columns):
Unnamed: 0    323870 non-null int64
Id            323870 non-null int64
year          323603 non-null float64
text          323870 non-null object
keywords      37845 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 12.4+ MB


In [87]:
%%time
fourgrams = open('./Processed_data/fourgrams.txt','r')
corpus = []
for line in fourgrams.readlines():
    corpus.append(line.strip().split())

CPU times: user 20.2 s, sys: 9.55 s, total: 29.8 s
Wall time: 33.5 s


In [89]:
data = pd.DataFrame({'text':corpus})
data.head()

,text
0,"[what, we, know, and, do_not, know_about, the,..."
1,"[abstract, it, have, be, well_appreciate, that..."
2,"[the, endocannabinoid_system, be, compose, of,..."
3,"[along, with, endocannabinoid, both, plant_der..."
4,"[because, no, cannabinoid, ligand, have, be, i..."


In [90]:
data.to_csv('./Processed_data/text_only.csv')